In [1]:
from STvAR_v3 import *
from STvARMetricObjects import *
from STvARFiniteDifference import *
from STvARAdvection import *
from STvARSymbolParsing import *
from STvARIndexing import*

In [2]:
DiffOrder = 4
DiffOrderKO = DiffOrder + 2
KOsigma = 0.3
dim = 3

In [3]:
stvar.gridvars = []
stvar.vartypes = []

PhiRe = stvar('PhiRe', gridvar = True, vartype = 'state_fab')
PhiIm = stvar('PhiIm', gridvar = True, vartype = 'state_fab')

Psi = stvar('Psi', gridvar = True, vartype = 'state_fab')

alpha = stvar('alpha', gridvar = True, vartype = 'state_fab')


In [4]:
Fullstring = ""

In [5]:
Fullstring += PhiRe.symb2isymb()
Fullstring += PhiRe.symb2isymb()
Fullstring += Psi.symb2isymb()
Fullstring += alpha.symb2isymb()

In [6]:
dDPhiRe_L = PhiRe.diff('dD', Accuracy = DiffOrder)

dDalpha_L = alpha.diff('dD', Accuracy = DiffOrder)

dDPsi_L = Psi.diff('dD', Accuracy = DiffOrder)
dDDPsi_LL = Psi.diff('dDD', Accuracy = DiffOrder)
dKODFullPsi = Psi.diff('dKODFull', Accuracy = DiffOrderKO)

In [7]:
Fullstring += dDPhiRe_L.symb2isymb()
Fullstring += dDalpha_L.symb2isymb()
Fullstring += dDPsi_L.symb2isymb()
Fullstring += dDDPsi_LL.symb2isymb()
Fullstring += dKODFullPsi.symb2isymb()

In [8]:
CovDPhiRe_L = stvarrank1('CovDPhi_L', dim = 4)
for i in range(dim):
    CovDPhiRe_L.expr[i] = dDPhiRe_L.symb[i]
CovDPhiRe_L.expr[3] = 0

In [9]:
Fullstring += CovDPhiRe_L.symb2expr()

In [10]:
g_LL = stvarrank2('g_LL', sym = 'diag01', dim = 4)

In [11]:
for i in range(3):
    for j in range(3):
        g_LL.expr[i][j] = (Psi.symb**4)*sp.eye(3)[i,j]
g_LL.expr[3][3] = -alpha.symb**2

In [12]:
Fullstring += g_LL.symb2expr()

In [13]:
invg_UU = inversemetric(g_LL)

In [14]:
Fullstring += invg_UU.symb2expr()

In [15]:
Vpotential = stvar('Vpotential')
Vpotential.expr = -2*(PhiRe.symb**2)*(1-1/2*(PhiRe.symb**2)) 

In [16]:
Fullstring += Vpotential.symb2expr()

In [17]:
T_LL = stvarrank2('T_LL', sym = 'sym01', dim = 4)
for a in range(4):
    for b in range(4):
        T_LL.expr[a][b] += (CovDPhiRe_L.symb[a]*CovDPhiRe_L.symb[b]) - g_LL.symb[a][b]*Vpotential.symb
        for c in range(4):
            for d in range(4):
                T_LL.expr[a][b] += -1/2*g_LL.symb[a][b]*(invg_UU.symb[c][d]*CovDPhiRe_L.symb[c]*CovDPhiRe_L.symb[d])

In [18]:
Fullstring += T_LL.symb2expr()

In [19]:
n_U = stvarrank1('n_U', dim = 4)
for i in range(3):
    n_U.expr[i] = 0
n_U.expr[3] = 1/alpha.symb

In [20]:
Fullstring += n_U.symb2expr()

In [21]:
rho = stvar('rho')
for a in range(4):
    for b in range(4):
        rho.expr += n_U.symb[a]*n_U.symb[b]*T_LL.symb[a][b]

In [22]:
Fullstring += rho.symb2expr()

In [24]:
LapPsi = stvar('LapPsi')
for i in range(3):
    for j in range(3):
        LapPsi.expr += sp.eye(3)[i,j]*dDDPsi_LL.symb[i][j]

In [25]:
Fullstring += LapPsi.symb2expr()

In [26]:
RHS_Psi = stvar('Psi', gridvar = True, addtolist = False, vartype = 'state_fab', varprefix = 'rhs_fab')
RHS_Psi.expr += LapPsi.symb + 2*sp.pi*(Psi.symb**5)*rho.symb

In [27]:
Fullstring += RHS_Psi.setisymb()

In [31]:
Fullstring = Fullstring.replace('state_fabIdx', 'Idx')

In [32]:
print(Fullstring)

        amrex::Real PhiRe = state_fab(i, j, k, Idx::PhiRe);

        amrex::Real PhiRe = state_fab(i, j, k, Idx::PhiRe);

        amrex::Real Psi = state_fab(i, j, k, Idx::Psi);

        amrex::Real alpha = state_fab(i, j, k, Idx::alpha);

        amrex::Real dDPhiRe_L_0 = ((2.0/3.0)*state_fab(i + 1, j, k, Idx::PhiRe) - 1.0/12.0*state_fab(i + 2, j, k, Idx::PhiRe) - 2.0/3.0*state_fab(i - 1, j, k, Idx::PhiRe) + (1.0/12.0)*state_fab(i - 2, j, k, Idx::PhiRe))/dx[0];
        amrex::Real dDPhiRe_L_1 = ((2.0/3.0)*state_fab(i, j + 1, k, Idx::PhiRe) - 1.0/12.0*state_fab(i, j + 2, k, Idx::PhiRe) - 2.0/3.0*state_fab(i, j - 1, k, Idx::PhiRe) + (1.0/12.0)*state_fab(i, j - 2, k, Idx::PhiRe))/dx[1];
        amrex::Real dDPhiRe_L_2 = ((2.0/3.0)*state_fab(i, j, k + 1, Idx::PhiRe) - 1.0/12.0*state_fab(i, j, k + 2, Idx::PhiRe) - 2.0/3.0*state_fab(i, j, k - 1, Idx::PhiRe) + (1.0/12.0)*state_fab(i, j, k - 2, Idx::PhiRe))/dx[2];

        amrex::Real dDalpha_L_0 = ((2.0/3.0)*state_fab(i + 1, j, k, Idx::alpha